# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [92]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [93]:
file = "../WeatherPy/CityWeather.csv"
weather_data = pd.read_csv(file)
weather_data.head()

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country
0,kavieng,-2.57,150.80,82.63,69,2,7.76,PG
1,bethel,41.37,-73.41,69.01,72,1,3.80,US
2,punta arenas,-53.15,-70.92,50.00,61,75,27.51,CL
3,pervomayskoye,43.92,46.71,69.12,60,0,10.40,RU
4,rikitea,-23.12,-134.97,72.14,80,54,11.68,PF


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [94]:
gmaps.configure(api_key=g_key)

In [118]:
locations = weather_data[["Lat", "Lng"]].astype(float)
humidity = weather_data["Humidity"].astype(float)

In [117]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)
fig

ValueError: weights must be of the same length as locations or None

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [97]:
perfect_weather = weather_data.loc[(weather_data["Max temp"] <80) & (weather_data["Max temp"] >70) & 
                                   (weather_data["Wind Speed"] <10) & (weather_data["Cloudiness"] ==0)]
perfect_weather = perfect_weather.dropna(how="any")
#perfect_weather = perfect_weather.reset_index()
perfect_weather.head()

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country
7,cape town,-33.93,18.42,77.00,31,0,6.93,ZA
26,bredasdorp,-34.53,20.04,73.40,49,0,6.93,ZA
68,castro,-24.79,-50.01,77.47,37,0,1.95,BR
129,bahia blanca,-38.72,-62.27,75.27,34,0,5.59,AR
163,saryagash,41.45,69.17,71.60,15,0,9.17,KZ


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [99]:
hotel_df["Hotel Name"] = ""

hotel_df.head()

,index,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
0,7,cape town,-33.93,18.42,77.00,31,0,6.93,ZA,
1,26,bredasdorp,-34.53,20.04,73.40,49,0,6.93,ZA,
2,68,castro,-24.79,-50.01,77.47,37,0,1.95,BR,
3,129,bahia blanca,-38.72,-62.27,75.27,34,0,5.59,AR,
4,163,saryagash,41.45,69.17,71.60,15,0,9.17,KZ,


In [108]:
params = {
    "radius": 5000,
    "types": "airport",
    "keyword": "nearest hotel",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]


    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    
    city_hotels = requests.get(base_url, params=params)
    city_hotels = city_hotels.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = city_hotels["results"][0]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "Not found"
        
hotel_df

,index,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
0,7,cape town,-33.93,18.42,77.00,31,0,6.93,ZA,Not found
1,26,bredasdorp,-34.53,20.04,73.40,49,0,6.93,ZA,Not found
2,68,castro,-24.79,-50.01,77.47,37,0,1.95,BR,Not found
3,129,bahia blanca,-38.72,-62.27,75.27,34,0,5.59,AR,Not found
4,163,saryagash,41.45,69.17,71.60,15,0,9.17,KZ,Not found
5,176,hami,42.80,93.45,71.33,23,0,6.53,CN,Not found
6,215,san quintin,30.48,-115.95,72.82,61,0,6.11,MX,Not found
7,269,neuquen,-38.95,-68.06,77.00,29,0,1.50,AR,Not found
8,328,kharan,28.58,65.42,79.00,17,0,3.04,PK,Kharan
9,437,kashi,39.45,75.98,71.60,28,0,1.77,CN,Not found


In [115]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [116]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))